In [ ]:
#default_exp website

# Website

> My static website

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
#export
from nb_cdk.iseninja import IseNinjaStack

from aws_cdk import (
    aws_apigateway,
    aws_cloudfront,
    aws_route53,
    aws_route53_targets,
    aws_s3,
    aws_s3_assets,
    aws_s3_deployment,
    core
)

In [ ]:
#export

class WebsiteStack(IseNinjaStack):
    """
    Stack containing resources for my personal website
    """
    def __init__(self, app: core.App, id: str, **kwargs) -> None:
        super().__init__(app, id, **kwargs)

        site_name = "IseNinjaHomePage"

        # Create S3 Bucket
        iseNinjaBucket = self.__build_site_bucket(site_name)
        core.CfnOutput(self, "IseNinjaBucketArn", value=iseNinjaBucket.bucket_arn)


        iseNinjaDistribution = self.__build_cloudfront_distribution(site_name, iseNinjaBucket)
        core.CfnOutput(self, "IseNinjaDistributionId", value=iseNinjaDistribution.distribution_id)

        # Create Route53 Mapping
        # ise.ninja -> CloudFrontDistribution
        aws_route53.ARecord(self, "IseNinjaRecord", record_name="ise.ninja", zone=self._hosted_zone, 
        target= aws_route53.RecordTarget.from_alias(aws_route53_targets.CloudFrontTarget(iseNinjaDistribution)))

        # Initial Deployment of our website resources
        aws_s3_deployment.BucketDeployment(self, "IseNinjaDeployment", 
          sources=[aws_s3_deployment.Source.asset('./website')],
          destination_bucket=iseNinjaBucket,
          distribution=iseNinjaDistribution,
          distribution_paths=["/*"])
        

    def __build_site_bucket(self, site_name: str):
        name = site_name + "Bucket"
        return aws_s3.Bucket(self, name,
          bucket_name="ise.ninja",
          website_index_document="index.html",
          website_error_document="error.html",
          public_read_access=True)

    def __build_cloudfront_distribution(self, site_name: str, bucket: aws_s3.Bucket):
        name = site_name + "Distribution"

        return aws_cloudfront.CloudFrontWebDistribution(self, name, 
          alias_configuration=aws_cloudfront.AliasConfiguration(
            acm_cert_ref = self._cert.certificate_arn,
            names = ["ise.ninja"],
            ssl_method=aws_cloudfront.SSLMethod.SNI,
            security_policy=aws_cloudfront.SecurityPolicyProtocol.TLS_V1_1_2016),
          origin_configs=[
             aws_cloudfront.SourceConfiguration(
                 custom_origin_source=aws_cloudfront.CustomOriginConfig(
                     domain_name=bucket.bucket_website_domain_name,
                     origin_protocol_policy=aws_cloudfront.OriginProtocolPolicy.HTTP_ONLY
                 ),
                 behaviors=[
                     aws_cloudfront.Behavior(
                         is_default_behavior=True
                     )
                 ]
             )
          ])


NameError: name 'IseNinjaStack' is not defined

In [ ]:
print("Initializing App...")
app = core.App()

print("Creating Website")
website = WebsiteStack(app, "PersonalWebsite")

print("Synthezing Application")
app.synth()

Initializing App...
Creating Website
Synthezing Application
